In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes pandas psutil

  Using cached unsloth-2025.12.9-py3-none-any.whl.metadata (65 kB)
Using cached unsloth-2025.12.9-py3-none-any.whl (376 kB)
Found existing installation: unsloth 2025.12.9
Uninstalling unsloth-2025.12.9:
  Successfully uninstalled unsloth-2025.12.9
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-c8380wyc/unsloth_7553248051d04ccb9842240e46ae7943
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-c8380wyc/unsloth_7553248051d04ccb9842240e46ae7943
  Resolved https://github.com/unslothai/unsloth.git to commit d83fbf67bbe1ca134cf510ea56099de2da3ec6f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.12.9-py3-none-any.whl size=382547 sha256=f8b9d3404f098426bdf3d7d6dfef2de6e466d78b4b9b5b961d3cfdcf6cf0736b
  Stored in directory: /tmp/pip-ephem-wheel-cache-whc83k2a/wheels/60/3e/1

# Installation & Setup

`Unsloth` for accelerated training (2x speed, 60% less memory), Hugging Face tools (`transformers`, `peft`, `trl`), and utility packages.
 A patch is applied to `psutil` to ensure compatibility with Google Colab's runtime environment

In [ ]:
import torch
import io
import psutil
import builtins
builtins.psutil = psutil

from unsloth import FastLanguageModel
from datasets import Dataset
import pandas as pd
import json
from transformers import TrainingArguments
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


#USE Qwen-2.5-3B

 "frozen" base model provides strong multilingual and reasoning capabilities, serving as the foundation for the specialized routing agent.

In [ ]:
#load model
max_seq_length = 2048
dtype = None
load_in_4bit = True

print("Loading Model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Loading Model...
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
csv_content = """id,instruction,input,output
1,"Extract intent","عايز اروح محطة الرمل من عند سموحة","{""intent"": ""routing"", ""origin"": ""سموحة"", ""destination"": ""محطة الرمل"", ""mode"": null, ""constraints"": []}"
2,"Extract intent","وديني سيدي جابر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سيدي جابر"", ""mode"": null, ""constraints"": []}"
3,"Extract intent","انا في جليم وعايز اروح سان ستيفانو","{""intent"": ""routing"", ""origin"": ""جليم"", ""destination"": ""سان ستيفانو"", ""mode"": null, ""constraints"": []}"
4,"Extract intent","عايز اروح القلعة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""قلعة قايتباي"", ""mode"": null, ""constraints"": []}"
5,"Extract intent","من العصافرة لبحري","{""intent"": ""routing"", ""origin"": ""العصافرة"", ""destination"": ""بحري"", ""mode"": null, ""constraints"": []}"
6,"Extract intent","فين شارع 45؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": null, ""constraints"": []}"
7,"Extract intent","من الكيلو 21 للموقف","{""intent"": ""routing"", ""origin"": ""الكيلو 21"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
8,"Extract intent","اركب مشروع ايه عشان اروح سموحة؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سموحة"", ""mode"": ""microbus"", ""constraints"": []}"
9,"Extract intent","عايز اركب ترام لحد فيكتوريا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""فيكتوريا"", ""mode"": ""tram"", ""constraints"": []}"
10,"Extract intent","فيه تمناية بتروح الحضرة من هنا؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الحضرة"", ""mode"": ""microbus_small"", ""constraints"": []}"
11,"Extract intent","عايز اروح المنشية بالاتوبيس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المنشية"", ""mode"": ""bus"", ""constraints"": []}"
12,"Extract intent","عايز اركب قطر لابوقير","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ابوقير"", ""mode"": ""train"", ""constraints"": []}"
13,"Extract intent","تمناية للكيلو 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": ""microbus_small"", ""constraints"": []}"
14,"Extract intent","عايز اروح المندرة بس بلاش الترام عشان بطيء","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المندرة"", ""mode"": null, ""constraints"": [""avoid_tram""]}"
15,"Extract intent","شوفلي طريق للموقف الجديد بس متمشيش من ع البحر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": [""avoid_corniche""]}"
16,"Extract intent","عايز ارخص مواصلة للمنشية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المنشية"", ""mode"": null, ""constraints"": [""cheapest""]}"
17,"Extract intent","اسرع طريق لسموحة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سموحة"", ""mode"": null, ""constraints"": [""fastest""]}"
18,"Extract intent","عايز اروح ال 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
19,"Extract intent","من ال 21 للموقف","{""intent"": ""routing"", ""origin"": ""الكيلو 21"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
20,"Extract intent","خدني عند 45","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": null, ""constraints"": []}"
21,"Extract intent","معايا 5 جيني بس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""budget_5_egp""]}"
22,"Extract intent","اركب توناية منين؟","{""intent"": ""general_info"", ""query_type"": ""location"", ""destination"": ""nearest_stop"", ""mode"": ""microbus_small"", ""constraints"": []}"
23,"Extract intent","من جليم لستانلي ازاي؟","{""intent"": ""routing"", ""origin"": ""جليم"", ""destination"": ""كوبري ستانلي"", ""mode"": null, ""constraints"": []}"
24,"Extract intent","سموحة للمندرة","{""intent"": ""routing"", ""origin"": ""سموحة"", ""destination"": ""المندرة"", ""mode"": null, ""constraints"": []}"
25,"Extract intent","المنشية؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المنشية"", ""mode"": null, ""constraints"": []}"
26,"Extract intent","التذكرة ب 10 جيني ولا اكتر؟","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""price_check""]}"
27,"Extract intent","اروح 45 ازاي من هنا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": null, ""constraints"": []}"
28,"Extract intent","فيه مشروع لل 21؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": ""microbus"", ""constraints"": []}"
29,"Extract intent","خدني توناية للعصافرة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""العصافرة"", ""mode"": ""microbus_small"", ""constraints"": []}"
30,"Extract intent","محطة الرمل، توناية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محطة الرمل"", ""mode"": ""microbus_small"", ""constraints"": []}"
31,"Extract intent","الموقف الجديد بياخد كام جيني؟","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": [""price_check""]}"
32,"Extract intent","من العجمي للكيلو","{""intent"": ""routing"", ""origin"": ""العجمي"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
33,"Extract intent","رايح ال 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
34,"Extract intent","طريق 45","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": null, ""constraints"": []}"
35,"Extract intent","عايز اروح شارع 30","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 30"", ""mode"": null, ""constraints"": []}"
36,"Extract intent","معيش غير 2 جيني","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""budget_2_egp""]}"
37,"Extract intent","اركب ايه عشان اروح 21؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
38,"Extract intent","توناية فاضية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": null, ""mode"": ""microbus_small"", ""constraints"": [""less_crowded""]}"
39,"Extract intent","مشروع رايح جاي ب 10 جيني","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": ""microbus"", ""constraints"": [""price_check""]}"
40,"Extract intent","عايز اروح ال 21 ضروري","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": [""fastest""]}"
41,"Extract intent","محرم بك ازاي؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محرم بك"", ""mode"": null, ""constraints"": []}"
42,"Extract intent","من سموحة لسبورتنج","{""intent"": ""routing"", ""origin"": ""سموحة"", ""destination"": ""سبورتنج"", ""mode"": null, ""constraints"": []}"
43,"Extract intent","عايز اروح ك م 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
44,"Extract intent","شكرا يا اسطى","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""thanks""]}"
45,"Extract intent","عايز اتمشى ع البحر في 45","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": ""walking"", ""constraints"": [""corniche_side""]}"
46,"Extract intent","وديني سيدي جابر محطة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محطة سيدي جابر"", ""mode"": null, ""constraints"": []}"
47,"Extract intent","سيدي جابر الشيخ","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سيدي جابر الشيخ"", ""mode"": null, ""constraints"": []}"
48,"Extract intent","عايز اروح مكتبة اسكندرية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مكتبة الإسكندرية"", ""mode"": null, ""constraints"": []}"
49,"Extract intent","من البيطاش للهانوفيل","{""intent"": ""routing"", ""origin"": ""البيطاش"", ""destination"": ""الهانوفيل"", ""mode"": null, ""constraints"": []}"
50,"Extract intent","عايز اروح جرين بلازا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""جرين بلازا"", ""mode"": null, ""constraints"": []}"
51,"Extract intent","من كارفور للموقف","{""intent"": ""routing"", ""origin"": ""كارفور"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
52,"Extract intent","ازاي اروح حديقة الحيوان؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""حديقة الحيوان"", ""mode"": null, ""constraints"": []}"
53,"Extract intent","طريق للنادي الاوليمبي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""النادي الاوليمبي"", ""mode"": null, ""constraints"": []}"
54,"Extract intent","عايز اروح سبورتنج ع البحر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سبورتنج"", ""mode"": null, ""constraints"": [""corniche_side""]}"
55,"Extract intent","وديني سبورتنج ع الترام","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سبورتنج"", ""mode"": null, ""constraints"": [""tram_side""]}"
56,"Extract intent","من الشاطبي لكامب شيزار","{""intent"": ""routing"", ""origin"": ""الشاطبي"", ""destination"": ""كامب شيزار"", ""mode"": null, ""constraints"": []}"
57,"Extract intent","عايز اروح الازاريطة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الازاريطة"", ""mode"": null, ""constraints"": []}"
58,"Extract intent","من كوم الدكة لمحطة مصر","{""intent"": ""routing"", ""origin"": ""كوم الدكة"", ""destination"": ""محطة مصر"", ""mode"": null, ""constraints"": []}"
59,"Extract intent","عايز اروح كرموز","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""كرموز"", ""mode"": null, ""constraints"": []}"
60,"Extract intent","من العامرية للموقف","{""intent"": ""routing"", ""origin"": ""العامرية"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
61,"Extract intent","عايز اروح برج العرب","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""برج العرب"", ""mode"": null, ""constraints"": []}"
62,"Extract intent","طريق للمطار القديم","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مطار النزهة"", ""mode"": null, ""constraints"": []}"
63,"Extract intent","عايز اروح المطار الجديد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مطار برج العرب"", ""mode"": null, ""constraints"": []}"
64,"Extract intent","من ابو قير للمندرة","{""intent"": ""routing"", ""origin"": ""ابو قير"", ""destination"": ""المندرة"", ""mode"": null, ""constraints"": []}"
65,"Extract intent","وديني عند فندق فلسطين","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""فندق فلسطين"", ""mode"": null, ""constraints"": []}"
66,"Extract intent","عايز اروح المنتزه","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""حدائق المنتزه"", ""mode"": null, ""constraints"": []}"
67,"Extract intent","من المعمورة البلد للمعمورة الشاطئ","{""intent"": ""routing"", ""origin"": ""المعمورة البلد"", ""destination"": ""المعمورة الشاطئ"", ""mode"": null, ""constraints"": []}"
68,"Extract intent","عايز اروح طوسون","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""طوسون"", ""mode"": null, ""constraints"": []}"
69,"Extract intent","من العصافرة بحر للمندرة","{""intent"": ""routing"", ""origin"": ""العصافرة بحر"", ""destination"": ""المندرة"", ""mode"": null, ""constraints"": []}"
70,"Extract intent","عايز اروح ميامي شارع اسكندر ابراهيم","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع اسكندر ابراهيم"", ""mode"": null, ""constraints"": []}"
71,"Extract intent","من خالد بن الوليد لجمال عبد الناصر","{""intent"": ""routing"", ""origin"": ""خالد بن الوليد"", ""destination"": ""جمال عبد الناصر"", ""mode"": null, ""constraints"": []}"
72,"Extract intent","وديني فيكتوريا ميدان الساعة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ميدان الساعة"", ""mode"": null, ""constraints"": []}"
73,"Extract intent","عايز اروح فلمنج","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""فلمنج"", ""mode"": null, ""constraints"": []}"
74,"Extract intent","من الظاهرية لباكوس","{""intent"": ""routing"", ""origin"": ""الظاهرية"", ""destination"": ""باكوس"", ""mode"": null, ""constraints"": []}"
75,"Extract intent","عايز اروح شدس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شدس"", ""mode"": null, ""constraints"": []}"
76,"Extract intent","طريق لزيزينيا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""زيزينيا"", ""mode"": null, ""constraints"": []}"
77,"Extract intent","عايز اروح ثروت","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ثروت"", ""mode"": null, ""constraints"": []}"
78,"Extract intent","من بوكلا لكليوباترا","{""intent"": ""routing"", ""origin"": ""بوكلا"", ""destination"": ""كليوباترا"", ""mode"": null, ""constraints"": []}"
79,"Extract intent","وديني مصطفى كامل","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مصطفى كامل"", ""mode"": null, ""constraints"": []}"
80,"Extract intent","عايز اروح رشدي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""رشدي"", ""mode"": null, ""constraints"": []}"
81,"Extract intent","من كفر عبده لميدان سانت جيني","{""intent"": ""routing"", ""origin"": ""كفر عبده"", ""destination"": ""ميدان سانت جيني"", ""mode"": null, ""constraints"": []}"
82,"Extract intent","عايز اروح الحضرة القبلية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الحضرة القبلية"", ""mode"": null, ""constraints"": []}"
83,"Extract intent","وديني الحضرة البحرية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الحضرة البحرية"", ""mode"": null, ""constraints"": []}"
84,"Extract intent","من النزهة لحديقة الورد","{""intent"": ""routing"", ""origin"": ""النزهة"", ""destination"": ""حديقة الورد"", ""mode"": null, ""constraints"": []}"
85,"Extract intent","عايز اروح محرم بك الرصافة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الرصافة"", ""mode"": null, ""constraints"": []}"
86,"Extract intent","طريق لغيط العنب","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""غيط العنب"", ""mode"": null, ""constraints"": []}"
87,"Extract intent","عايز اروح راغب","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""راغب"", ""mode"": null, ""constraints"": []}"
88,"Extract intent","من كرموز للعامود","{""intent"": ""routing"", ""origin"": ""كرموز"", ""destination"": ""عامود السواري"", ""mode"": null, ""constraints"": []}"
89,"Extract intent","وديني عند كوبري التاريخ","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""كوبري التاريخ"", ""mode"": null, ""constraints"": []}"
90,"Extract intent","عايز اروح القباري","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""القباري"", ""mode"": null, ""constraints"": []}"
91,"Extract intent","من الورديان للمكس","{""intent"": ""routing"", ""origin"": ""الورديان"", ""destination"": ""المكس"", ""mode"": null, ""constraints"": []}"
92,"Extract intent","عايز اروح الدخيلة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الدخيلة"", ""mode"": null, ""constraints"": []}"
93,"Extract intent","طريق لبيانكي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""بيانكي"", ""mode"": null, ""constraints"": []}"
94,"Extract intent","عايز اروح شهر العسل","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شهر العسل"", ""mode"": null, ""constraints"": []}"
95,"Extract intent","من ابو يوسف للكيلو","{""intent"": ""routing"", ""origin"": ""ابو يوسف"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
96,"Extract intent","وديني عند بوابة 8","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""بوابة 8"", ""mode"": null, ""constraints"": []}"
97,"Extract intent","عايز اروح مارينا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مارينا"", ""mode"": null, ""constraints"": []}"
98,"Extract intent","من سيدي كرير لبرج العرب","{""intent"": ""routing"", ""origin"": ""سيدي كرير"", ""destination"": ""برج العرب"", ""mode"": null, ""constraints"": []}"
99,"Extract intent","عايز اروح ستاد الجيش","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ستاد برج العرب"", ""mode"": null, ""constraints"": []}"
100,"Extract intent","طريق للمنطقة الصناعية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المنطقة الصناعية"", ""mode"": null, ""constraints"": []}"
101,"Extract intent","عايز اروح الكورنيش","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""كورنيش البحر"", ""mode"": null, ""constraints"": []}"
102,"Extract intent","وديني شارع سوتر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع سوتر"", ""mode"": null, ""constraints"": []}"
103,"Extract intent","من المجمع النظري لمجمع الكليات الطبية","{""intent"": ""routing"", ""origin"": ""المجمع النظري"", ""destination"": ""مجمع الكليات الطبية"", ""mode"": null, ""constraints"": []}"
104,"Extract intent","عايز اروح شارع فؤاد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع فؤاد"", ""mode"": null, ""constraints"": []}"
105,"Extract intent","من النبي دانيال لمحطة الرمل","{""intent"": ""routing"", ""origin"": ""شارع النبي دانيال"", ""destination"": ""محطة الرمل"", ""mode"": null, ""constraints"": []}"
106,"Extract intent","وديني شارع صفية زغلول","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع صفية زغلول"", ""mode"": null, ""constraints"": []}"
107,"Extract intent","عايز اروح شارع السلطان حسين","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع السلطان حسين"", ""mode"": null, ""constraints"": []}"
108,"Extract intent","طريق لشارع المشير","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع المشير"", ""mode"": null, ""constraints"": []}"
109,"Extract intent","عايز اروح سيدي جابر ع البحر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سيدي جابر"", ""mode"": null, ""constraints"": [""corniche_side""]}"
110,"Extract intent","من كليوباترا حمامات لكليوباترا الصغرى","{""intent"": ""routing"", ""origin"": ""كليوباترا حمامات"", ""destination"": ""كليوباترا الصغرى"", ""mode"": null, ""constraints"": []}"
111,"Extract intent","وديني سان ستيفانو مول","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سان ستيفانو جراند بلازا"", ""mode"": null, ""constraints"": []}"
112,"Extract intent","عايز اروح جليم باي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""جليم باي"", ""mode"": null, ""constraints"": []}"
113,"Extract intent","من ستانلي لكوبري الجامعة","{""intent"": ""routing"", ""origin"": ""كوبري ستانلي"", ""destination"": ""كوبري الجامعة"", ""mode"": null, ""constraints"": []}"
114,"Extract intent","عايز اروح القائد ابراهيم","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مسجد القائد ابراهيم"", ""mode"": null, ""constraints"": []}"
115,"Extract intent","طريق للمرسي ابو العباس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""مسجد المرسي ابو العباس"", ""mode"": null, ""constraints"": []}"
116,"Extract intent","وديني الانفوشي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الانفوشي"", ""mode"": null, ""constraints"": []}"
117,"Extract intent","عايز اروح راس التين","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""راس التين"", ""mode"": null, ""constraints"": []}"
118,"Extract intent","من السيالة للجمرك","{""intent"": ""routing"", ""origin"": ""السيالة"", ""destination"": ""الجمرك"", ""mode"": null, ""constraints"": []}"
119,"Extract intent","عايز اروح المنشية الصغرى","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المنشية الصغرى"", ""mode"": null, ""constraints"": []}"
120,"Extract intent","طريق للسبع بنات","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""السبع بنات"", ""mode"": null, ""constraints"": []}"
121,"Extract intent","عايز اروح شارع فرنسا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع فرنسا"", ""mode"": null, ""constraints"": []}"
122,"Extract intent","وديني سوق ليبيا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""سوق ليبيا"", ""mode"": null, ""constraints"": []}"
123,"Extract intent","من العامود لكوم الشقافة","{""intent"": ""routing"", ""origin"": ""عامود السواري"", ""destination"": ""مقابر كوم الشقافة"", ""mode"": null, ""constraints"": []}"
124,"Extract intent","عايز اروح الباب الجديد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الباب الجديد"", ""mode"": null, ""constraints"": []}"
125,"Extract intent","طريق لقناة السويس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع قناة السويس"", ""mode"": null, ""constraints"": []}"
126,"Extract intent","عايز اروح كوبري محرم بك","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""كوبري محرم بك"", ""mode"": null, ""constraints"": []}"
127,"Extract intent","من نادي الصيد للمطار","{""intent"": ""routing"", ""origin"": ""نادي الصيد"", ""destination"": ""مطار النزهة"", ""mode"": null, ""constraints"": []}"
128,"Extract intent","وديني عزبة سعد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""عزبة سعد"", ""mode"": null, ""constraints"": []}"
129,"Extract intent","عايز اروح انطونيادس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""حدائق انطونيادس"", ""mode"": null, ""constraints"": []}"
130,"Extract intent","طريق لسموحة نادي سموحة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""نادي سموحة"", ""mode"": null, ""constraints"": []}"
131,"Extract intent","عايز اروح كوبري كليوباترا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""كوبري كليوباترا"", ""mode"": null, ""constraints"": []}"
132,"Extract intent","من كوبري الابراهيمية لسبورتنج","{""intent"": ""routing"", ""origin"": ""كوبري الابراهيمية"", ""destination"": ""سبورتنج"", ""mode"": null, ""constraints"": []}"
133,"Extract intent","وديني شارع بورسعيد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع بورسعيد"", ""mode"": null, ""constraints"": []}"
134,"Extract intent","عايز اروح شارع جمال عبد الناصر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع جمال عبد الناصر"", ""mode"": null, ""constraints"": []}"
135,"Extract intent","طريق لشارع الملك حفني","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع الملك حفني"", ""mode"": null, ""constraints"": []}"
136,"Extract intent","عايز اروح العصافرة قبلي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""العصافرة قبلي"", ""mode"": null, ""constraints"": []}"
137,"Extract intent","من سيدي بشر ترام لسيدي بشر بحر","{""intent"": ""routing"", ""origin"": ""سيدي بشر ترام"", ""destination"": ""سيدي بشر بحر"", ""mode"": null, ""constraints"": []}"
138,"Extract intent","وديني بير مسعود","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""بير مسعود"", ""mode"": null, ""constraints"": []}"
139,"Extract intent","عايز اروح ميامي الجديدة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ميامي الجديدة"", ""mode"": null, ""constraints"": []}"
140,"Extract intent","طريق للراس السودا","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الراس السودا"", ""mode"": null, ""constraints"": []}"
141,"Extract intent","عايز اروح شارع مصطفى كامل","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع مصطفى كامل"", ""mode"": null, ""constraints"": []}"
142,"Extract intent","من الفلكي لفيكتوريا","{""intent"": ""routing"", ""origin"": ""الفلكي"", ""destination"": ""فيكتوريا"", ""mode"": null, ""constraints"": []}"
143,"Extract intent","وديني السيوف شماعة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""السيوف شماعة"", ""mode"": null, ""constraints"": []}"
144,"Extract intent","عايز اروح الساعة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ميدان الساعة"", ""mode"": null, ""constraints"": []}"
145,"Extract intent","طريق للحجر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الحجر"", ""mode"": null, ""constraints"": []}"
146,"Extract intent","عايز اروح العوايد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""العوايد"", ""mode"": null, ""constraints"": []}"
147,"Extract intent","من كوبري الناموس للموقف","{""intent"": ""routing"", ""origin"": ""كوبري الناموس"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
148,"Extract intent","وديني ابيس","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ابيس"", ""mode"": null, ""constraints"": []}"
149,"Extract intent","عايز اروح قرية الاسد","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""قرية الاسد"", ""mode"": null, ""constraints"": []}"
150,"Extract intent","طريق للعامرية","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""العامرية"", ""mode"": null, ""constraints"": []}"
151,"Extract intent","التذكرة للمعمورة بكام يا اسطى؟","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": ""المعمورة"", ""mode"": null, ""constraints"": [""price_check""]}"
152,"Extract intent","الطريق للموقف بياخد وقت قد ايه؟","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": [""time_check""]}"
153,"Extract intent","هو الترام شغال النهاردة؟","{""intent"": ""general_info"", ""origin"": null, ""destination"": null, ""mode"": ""tram"", ""constraints"": [""status_check""]}"
154,"Extract intent","تسلم يا غالي شكرا","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""thanks""]}"
155,"Extract intent","صباح الفل","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""greeting""]}"
156,"Extract intent","وديني العجمي الهانوفيل","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""العجمي الهانوفيل"", ""mode"": null, ""constraints"": []}"
157,"Extract intent","عايز اروح ميامي الجديدة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""ميامي الجديدة"", ""mode"": null, ""constraints"": []}"
158,"Extract intent","من سيدي بشر ترام لسيدي بشر بحر","{""intent"": ""routing"", ""origin"": ""سيدي بشر ترام"", ""destination"": ""سيدي بشر بحر"", ""mode"": null, ""constraints"": []}"
159,"Extract intent","العصافره","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""العصافرة"", ""mode"": null, ""constraints"": []}"
160,"Extract intent","عايز اروح ألموقف","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
161,"Extract intent","المحطة فين؟","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محطة مصر"", ""mode"": null, ""constraints"": []}"
162,"Extract intent","فين موقف ميكروباصات سموحة","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": ""موقف سموحة"", ""mode"": ""microbus"", ""constraints"": [""location_check""]}"
163,"Extract intent","اجرة المشروع كام","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": ""microbus"", ""constraints"": [""price_check""]}"
164,"Extract intent","هوصل في كام دقيقة؟","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""time_check""]}"
165,"Extract intent","شكرا","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""thanks""]}"
166,"Extract intent","مساء الخير","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""greeting""]}"
167,"Extract intent","بكام","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""price_check""]}"
168,"Extract intent","وقت قد ايه","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""time_check""]}"
169,"Extract intent","وديني عند ال 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
170,"Extract intent","عايز اروح ك م 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
171,"Extract intent","شارع 45","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": null, ""constraints"": []}"
172,"Extract intent","عايز اروح 45","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""شارع 45"", ""mode"": null, ""constraints"": []}"
173,"Extract intent","الموقف","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الموقف الجديد"", ""mode"": null, ""constraints"": []}"
174,"Extract intent","من الموقف","{""intent"": ""routing"", ""origin"": ""الموقف الجديد"", ""destination"": null, ""mode"": null, ""constraints"": []}"
175,"Extract intent","فين المحطة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محطة مصر"", ""mode"": null, ""constraints"": []}"
176,"Extract intent","عايز اروح محطة القطر","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محطة مصر"", ""mode"": null, ""constraints"": []}"
177,"Extract intent","من العصافرة قبلي لبحري","{""intent"": ""routing"", ""origin"": ""العصافرة قبلي"", ""destination"": ""بحري"", ""mode"": null, ""constraints"": []}"
178,"Extract intent","وديني المندرة قبلي","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""المندرة قبلي"", ""mode"": null, ""constraints"": []}"
179,"Extract intent","سعر التذكرة","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""price_check""]}"
180,"Extract intent","بتاخد وقت اد ايه","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""time_check""]}"
181,"Extract intent","السلام عليكم","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""greeting""]}"
182,"Extract intent","ازيك","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""greeting""]}"
183,"Extract intent","ميرسي","{""intent"": ""other"", ""origin"": null, ""destination"": null, ""mode"": null, ""constraints"": [""thanks""]}"
184,"Extract intent","عايز اروح موقف محرم بك","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""موقف محرم بك"", ""mode"": null, ""constraints"": []}"
185,"Extract intent","من موقف السيوف","{""intent"": ""routing"", ""origin"": ""موقف السيوف"", ""destination"": null, ""mode"": null, ""constraints"": []}"
186,"Extract intent","المحطه","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""محطة مصر"", ""mode"": null, ""constraints"": []}"
187,"Extract intent","المنتزة","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""حدائق المنتزه"", ""mode"": null, ""constraints"": []}"
188,"Extract intent","وديني ال 21","{""intent"": ""routing"", ""origin"": ""current_location"", ""destination"": ""الكيلو 21"", ""mode"": null, ""constraints"": []}"
189,"Extract intent","التذكرة بكام","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""price_check""]}"
190,"Extract intent","المسافة قد ايه","{""intent"": ""general_info"", ""origin"": ""current_location"", ""destination"": null, ""mode"": null, ""constraints"": [""time_check""]}"
"""
df = pd.read_csv(io.StringIO(csv_content))

In [ ]:
#  System Prompt (Strict Rules for General Info & Ambiguity)
system_prompt = """You are an expert Routing-NLU for Alexandria, Egypt.
Your task is to extract user intent and entities from Egyptian dialect into strict JSON format.

Use this exact schema for ALL intents:
{
  "intent": "routing | general_info | other",
  "origin": "string (or 'current_location')",
  "destination": "string (or null)",
  "mode": "microbus | microbus_small | tram | bus | train | walking | null",
  "constraints": ["price_check" | "time_check" | "status_check" | "location_check" | "greeting" | "thanks" | other routing constraints]
}

Important Rules & Normalization:
1. **Terminology:**
   - "مشروع" = "microbus", "توناية" = "microbus_small".
   - "جيني", "جنيه", "جني", "LE" = "EGP".

2. **Ambiguity Resolution:**
   - "الموقف" alone -> "الموقف الجديد".
   - "موقف [Place]" -> "موقف [Place]" (A specific stop).
   - "المحطة" alone -> "محطة مصر".
   - "العصافره" -> "العصافرة", "المنتزة" -> "المنتزه".
   - "45" -> "شارع 45", "ال 21" -> "الكيلو 21".

3. **Intents:**
   - If asking about price ("بكام", "تذكرة") -> intent: "general_info", constraints: ["price_check"].
   - If asking about time ("وقت", "كام دقيقة") -> intent: "general_info", constraints: ["time_check"].
   - Greetings ("صباح", "سلام", "ازيك") -> intent: "other", constraints: ["greeting"].

Output ONLY the JSON object. Do not add any explanation."""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{output}<|im_end|>"
        texts.append(text)
    return { "text" : texts }

dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/190 [00:00<?, ? examples/s]

## LoRA Adapter Configuration
Configures **Low-Rank Adaptation (LoRA)** parameters. By targeting specific model modules (`q_proj`, `k_proj`, etc.) and using a rank of 16, enable efficient fine-tuning that adapts the model to the new task without modifying the massive pre-trained weights

In [ ]:
# Step LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


##Model Training (Fine-Tuning)
Executes the fine-tuning process using the `SFTTrainer`. The model is trained for **70 steps** (approx. 3 epochs) with a learning rate of `2e-4`. This phase adapts the model to understand local nuances and output structured JSON data accurately

In [ ]:

# Traning
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 70,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 5,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "qwen_alex_routing_v2",
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = training_args,
)

print(" Starting Training (Version 2 - Robust)...")
trainer_stats = trainer.train()
print("Training Finished!")

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/190 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


 Starting Training (Version 2 - Robust)...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 190 | Num Epochs = 3 | Total steps = 70
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,2.611200
10,1.727900
15,1.023500
20,0.409900
25,0.129800
30,0.081700
35,0.067400
40,0.070800
45,0.072900
50,0.065600


Training Finished!


In [ ]:
FastLanguageModel.for_inference(model)

def predict(user_query):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    # Greedy Decoding
    outputs = model.generate(
        input_ids = inputs,
        max_new_tokens = 128,
        use_cache = True,
        do_sample = False, # rather than temperature=0.0
    )
    response = tokenizer.batch_decode(outputs)
    final_json = response[0].split('<|im_start|>assistant')[-1].replace('<|im_end|>', '').strip()
    return final_json

print("\n--- Final Robust Test ---")
test_cases = [
    "وديني العجمي الهانوفيل",
    "من سيدس جابر للعصافره ازاي؟",
    " من سموحة وديني الموقف",
    "التذكرة للمعمورة بكام",
    "صباح الفل يا باشا",
    "الطريق ياخد وقت اد ايه",
    "الطريق من سيدي بشر لسيدي جابر اي",
    "اركب من موقف سيدي جابر اي يوديني عزبة سعد معيش غير 6 جني"
]

for query in test_cases:
    print(f"\n اليوزر: {query}")
    print(f" الموديل: {predict(query)}")




--- Final Robust Test ---

 اليوزر: وديني العجمي الهانوفيل
 الموديل: {"intent": "routing", "origin": "current_location", "destination": "العجمي الهانوفيل", "mode": null, "constraints": []}

 اليوزر: من سيدس جابر للعصافره ازاي؟
 الموديل: {"intent": "routing", "origin": "سيدس جابر", "destination": "العصافرة", "mode": null, "constraints": []}

 اليوزر:  من سموحة وديني الموقف
 الموديل: {"intent": "routing", "origin": "سموحة", "destination": null, "mode": null, "constraints": ["current_location"]}

 اليوزر: التذكرة للمعمورة بكام
 الموديل: {"intent": "general_info", "origin": "current_location", "destination": "المعمورة", "mode": null, "constraints": ["price_check"]}

 اليوزر: صباح الفل يا باشا
 الموديل: {"intent": "other", "origin": null, "destination": null, "mode": null, "constraints": ["greeting"]}

 اليوزر: الطريق ياخد وقت اد ايه
 الموديل: {"intent": "general_info", "origin": "current_location", "destination": null, "mode": null, "constraints": ["time_check"]}

 اليوزر: الطريق من سيدي 

In [ ]:
#Save on Deive
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/Qwen_Egyptian_Routing_Model_V2"

print(f"Saving V2 to {output_dir}...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("Saved Successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving V2 to /content/drive/MyDrive/Qwen_Egyptian_Routing_Model_V2...
Saved Successfully!


In [ ]:
print(trainer_stats)

TrainOutput(global_step=70, training_loss=0.4620523567710604, metrics={'train_runtime': 415.8284, 'train_samples_per_second': 1.347, 'train_steps_per_second': 0.168, 'total_flos': 4351279042805760.0, 'train_loss': 0.4620523567710604, 'epoch': 2.9263157894736844})


adapter_model.safetensors :The "Brain" Update.
This contains the fine-tuned LoRA weights (the specific knowledge about "Mashrou", "Mawkif", and prices)

It is loaded on top of the base Qwen model.adapter_config.json: Configuration. Tells the loading library (Peft/Unsloth) how to attach the adapters to the base model (Rank, Alpha, Target Modules)

tokenizer.json & vocab.json :The Translator. Critical for the Egyptian dialect. These files ensure the model correctly tokenizes words like "توناية" or "العصافرة" instead of breaking them into nonsense.special_tokens_map.json: Control Tokens  Defines special markers like `<

generation_config.json :Inference Settings Default parameters for text generation